## Imports

In [1]:
import os
from PIL import Image
from glob import glob
import scipy.misc
from random import randrange, randint, uniform
import copy
import math 
%matplotlib inline

## Helper Functions / Classes

In [2]:
class ImageGen():
    def __init__(self):
        self.base_dir = os.getcwd()
        self.data_dir = os.path.join(self.base_dir, 'data')
        self.crop_dir = os.path.join(self.data_dir, 'crops')
        self.bg_dir = os.path.join(self.data_dir, 'bgs')
        self.gen_dir = os.path.join(self.data_dir, 'gen')
        self.ori_dir = os.path.join(self.data_dir, 'ori')
        self.ori_csv = os.path.join(self.ori_dir, 'orientations.csv')
        self.training_list = os.path.join(self.gen_dir, 'training_list.txt')
        
        # temp
        self.counter = 0
        
        # misc
        self.min_alt = .5 # .5 meters min altitude for image generation
        self.max_alt = 3  # 3 meters max altitude for image generation
        self.class_ids = {'roomba':'0', 'obstacle':'1'}      
        self.camera_fov = 90 * math.pi / 180
        
        # crop images
        self.crop_width = 512
        self.get_crops()
        self.roomba_meter_ratio = 13.39 * 2.54 / 100
        
        # background images
        self.pixels_per_meter = 600
        self.bg_width = 1280
        self.bg_height = 720
        self.maximum_meters_seen = self.max_alt * 2 * math.tan(self.camera_fov / 2)
        self.get_bgs()
    
    def get_crops(self):
        """
        reads in and resizes cropped images into the self.crop_imgs list
        """
        os.chdir(self.crop_dir)
        self.crop_filenames = sorted(glob("*"))
        self.crop_imgs = [Image.open(filename) for filename in self.crop_filenames]
        self.crop_imgs = [self.resize_img(image, self.crop_width) for image in self.crop_imgs]
        os.chdir(self.base_dir)
        
    def get_bgs(self):
        """
        reads in and resizes background images the self.bg_imgs list
        """
        os.chdir(self.bg_dir)
        self.bg_filenames = sorted(glob("*"))
        self.bg_imgs = [Image.open(filename) for filename in self.bg_filenames]
#         self.bg_imgs = [image.resize((self.bg_width, self.bg_height), resample=Image.BILINEAR)
#                         for image in self.bg_imgs]
        os.chdir(self.base_dir)
        
    def resize_img(self, img, desired_width):
        """
        resizes target img while preserving width 
        
        Args:
            img (PIL Image): the image to be resized
            desired_width: the desired width for the resized image
            
        Returns:
            resized_img (PIL Image): the resized image
        """
        width, height = img.size
        ratio = width / height
        desired_height = int(desired_width / ratio)
        resized_img = img.resize((desired_width, desired_height), resample=Image.BILINEAR)
        return resized_img
    
    def crop_center(self, img, width, height):
        left  = int((img.width / 2) - (width / 2))
        right = left + width
        upper = int((img.height / 2) - (height / 2))
        lower = upper + height
        return img.crop((left, upper, right, lower))
    
    def gen_ori(self, file_id):
        """
        generates an image and its corresponding annotation files
            
        Args:
            file_id (string): the filename that should be saved
        """
        # first determine the altitude at which to generate
        altitude = 1 #uniform(self.min_alt, self.max_alt)
        zoom = ((self.bg_width/ (self.pixels_per_meter * self.maximum_meters_seen)) *
                self.max_alt / altitude)
        
        # prepare the fg (roomba) image
        fg_rot = self.counter #randint(0,360)
        self.counter += 1
        fg_rad = fg_rot * (math.pi / 180)
        xhat = math.cos(fg_rad)
        yhat = math.sin(fg_rad)
        fg = self.prepare_fg(zoom, fg_rot)
        
        # prepare the bg (gym floor) image
        bg_rot = 0 #randint(0,360)
        bg = self.prepare_bg(zoom, bg_rot)
        
        # generate and save the image
        x = randint(0, bg.width-fg.width)
        y = randint(0, bg.height-fg.height)
        try: # sometimes it breaks when very partiular things go wrong, this is
             # to catch those small but unlikely errors that can occure
            bg.paste(fg, (x, y), fg)
            bg = bg.crop((x, y, x + fg.width, y + fg.height)) # crop out roomba from background
        except:
            print("overlay failed: ", fg.size, bg.size, x, y)
            return
        img_filepath = os.path.join(self.ori_dir, "{:06}.png".format(file_id)) 
        bg.save(img_filepath)
        
        # generate corresponding annotation files
        self.append_to_file(self.ori_csv,
                            "{},{},{},{}".format(img_filepath,
                                                 fg_rot,
                                                 bg_rot,
                                                 altitude))
        
    def gen_img_ann(self, file_id):
        """
        generates an image and its corresponding annotation files
            
        Args:
            file_id (string): the filename that should be saved
        """
        # first determine the altitude at which to generate
        altitude = uniform(self.min_alt, self.max_alt)
        zoom = ((self.bg_width/ (self.pixels_per_meter * self.maximum_meters_seen)) *
                self.max_alt / altitude)
        
        # prepare the fg (roomba) image
        fg_rot = randint(0,360)
        fg_rad = fg_rot * (math.pi / 180)
        xhat = math.cos(fg_rad)
        yhat = math.sin(fg_rad)
        fg = self.prepare_fg(zoom, fg_rot)
        
        # prepare the bg (gym floor) image
        bg_rot = randint(0,360)
        bg = self.prepare_bg(zoom, bg_rot)
        
        # generate and save the image
        x = randint(0, bg.width-fg.width)
        y = randint(0, bg.height-fg.height)
        try: # sometimes it breaks when very partiular things go wrong, this is
             # to catch those small but unlikely errors that can occure
            bg.paste(fg, (x, y), fg)
        except:
            print("overlay failed: ", fg.size, bg.size, x, y)
            return
        img_filepath = os.path.join(self.gen_dir, "{:06}.png".format(file_id)) 
        txt_filepath = os.path.join(self.gen_dir, "{:06}.txt".format(file_id))
        bg.save(img_filepath)
        
        # generate corresponding annotation files
        self.remove_file_contents(txt_filepath)
        self.append_to_file(txt_filepath, self.class_ids['roomba'] + ' ' + self.yolo_bbox((x, x + fg.width, y, y + fg.height)))
        self.append_to_file(self.training_list, img_filepath)
    
    def prepare_fg(self, zoom, rot):
        fg = copy.deepcopy(self.crop_imgs[randint(0,1)]) # overhead picture
        fg = self.resize_img(fg, int(self.pixels_per_meter *
                                     self.roomba_meter_ratio * 
                                     zoom))
        fg = fg.rotate(rot, expand=True, resample=Image.BILINEAR)
        fg = fg.crop(fg.getbbox()) # remove any extra transparent pixels
        return fg
    
    def prepare_bg(self, zoom, rot):
        bg = copy.deepcopy(self.bg_imgs[2]) # 8mx8m competition grid
        bg = bg.crop((randint(0,600), randint(0,600), bg.width, bg.height)) # translate grid
        bg = bg.rotate(rot, expand=True, resample=Image.BILINEAR)
        bg = self.crop_center(bg,
                              int(self.bg_width / zoom) + 1, # +1 to fix transparent edges
                              int(self.bg_height / zoom) + 1)
        bg = self.resize_img(bg, self.bg_width)
        return bg
        
    def overlay_imgs(self, fg, bg, x, y):
        """
        overlays a foreground image onto a background image modified in place
        
        Args:
            fg (PIL Image): the foreground image
            bg (PIL Image): the background image
            x (int): the x position of the upper left corner to overlay image
            y (int): the y position of the upper left corner to overlay image
            
        Returns:
            True:  successful overlay
            False: unsuccessful overlay
        """
        try:
            bg.paste(fg, (x, y), fg)
            return True
        except:
            return False
        
    def yolo_bbox(self, box):
        """
        converts a regular bbox to a yolo bbox given the bg image dimensions
        
        Args;
            bbox (4 tuple): (left, right, upper, lower)
        """
        dw = 1 / self.bg_width
        dh = 1 / self.bg_height
        x = (box[0] + box[1]) / 2.0
        y = (box[2] + box[3]) / 2.0
        w = box[1] - box[0]
        h = box[3] - box[2]
        x = x * dw
        w = w * dw
        y = y * dh
        h = h * dh
        return ' '.join(map(str, (x, y, w, h)))
    
    def remove_file_contents(self, filepath):
        """
        clears the file located at filepath of all its contents
        
        Args:
            filepath (string): the path to the file to be cleared
            
        Returns:
            None
        """
        open(filepath, 'w').close()

    def append_to_file(self, filepath, data):
        """
        append data to file located at filepath
        
        Args:
            filepath (string): the file to be appended to
            data (string or string castable): the data to be appended
            
        Returns:
            None
        """
        with open(filepath, 'a') as f:
            f.write(str(data) + '\n')
            
    def generate_data(self, n):
        """
        generates n images and annotations
        """
        self.remove_file_contents(self.training_list)
        for i in range(n):
            self.gen_img_ann(i)
            
    def generate_orientation_data(self, n):
        self.remove_file_contents(self.ori_csv)
        self.append_to_file(self.ori_csv, "filepath,roomba_rotation,grid_rotation,drone_altitude")
        for i in range(n):
            self.gen_ori(i)

In [3]:
imageGen = ImageGen()

In [4]:
imageGen.generate_orientation_data(360)

In [ ]:
imageGen.generate_data(20)

In [ ]:
fg = imageGen.crop_imgs[1]
bg = imageGen.bg_imgs[2]

In [ ]:
i = 0
while i < 10:
    test = imageGen.overlay_imgs(fg, bg, x, y, rot, zoom)
    if test != None:
        test.save("/tmp/test{}.png".format(i))
        i += 1

In [ ]:
%%timeit
imageGen.resize_img(imageGen.crop_imgs[0], 256);

In [ ]:
test_img = test_img.rotate(randrange(0,360), expand=True)

In [ ]:
fg_copy = copy.deepcopy(imageGen.crop_imgs[1])
fg_copy.size

In [ ]:
fg_copy = fg_copy.rotate(45, expand=True)
fg_copy.size

## Get the cropped images

In [ ]:
self.maximum_meters_seen = self.max_alt * 2 * math.tan(self.camera_fov / 2)

In [ ]:
max_alt = 3

In [ ]:
math.tan(45 * math.pi / 180)

## Get background images

## Generate image

In [ ]:
os.chdir(CROP_DIR)



